<a href="https://colab.research.google.com/github/JakeOh/202105_itw_bd26/blob/main/lab_ml/ml03_train_test_set_scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 훈련 셋과 테스트 셋을 나누는 방법
    * 순차적 추출(sequential sampling)
    * 임의 추출(random sampling)
    * 층화 추출(stratified sampling)
* 분류 모델 평가 지표
    * 정확도(accuracy)
    * 정밀도(precision)
    * 재현율(recall)
    * F1-score
* 특성 스케일링(feature scaling)
    * 표준화(standardization)
    * 정규화(normalization)

# 필요한 모듈 import

In [1]:
import numpy as np  # ndarray 함수, 메서드
import pandas as pd  # Series, DataFrame 함수, 메서드
import matplotlib.pyplot as plt  # 시각화
import seaborn as sns  # 시각화

# Machine Learning
from sklearn.neighbors import KNeighborsClassifier  # KNN 분류기(모델, 알고리즘)
from sklearn.model_selection import train_test_split  # 훈련/테스트 셋 분리 함수
from sklearn.metrics import confusion_matrix, classification_report  # 모델 평가 지표
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # 특성 스케일링

# 데이터 준비

* fish.csv 파일: 어종, 생선 특성(무게, 길이, ...)
    * 문제를 간단히 하기 위해서, 어종 2개(Bream, Smelt)와 특성 2개(Weight, Length) 사용

In [2]:
fish_csv = 'https://github.com/rickiepark/hg-mldl/raw/master/fish.csv'

In [3]:
fish = pd.read_csv(fish_csv)  # CSV 파일 ---> DataFrame 생성

In [4]:
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [5]:
fish.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Species   159 non-null    object 
 1   Weight    159 non-null    float64
 2   Length    159 non-null    float64
 3   Diagonal  159 non-null    float64
 4   Height    159 non-null    float64
 5   Width     159 non-null    float64
dtypes: float64(5), object(1)
memory usage: 7.6+ KB


In [7]:
# 타겟 벡터(1차원 배열)
# 어종인 Bream 또는 Smelt인 Species 컬럼을 선택해서 numpy 배열로 변환(to_numpy() 메서드도 동일)
y = fish.loc[fish.Species.isin(['Bream', 'Smelt']), 'Species'].values
y

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)

In [8]:
y.shape  #> 원소가 49개인 1차원 배열

(49,)

In [10]:
# 타겟의 레이블 개수
np.unique(y, return_counts=True)

(array(['Bream', 'Smelt'], dtype=object), array([35, 14]))

In [13]:
# 특성 행렬(feature matrix)
# 어종이 Bream 또는 Smelt인 Weight 컬럼과 Length 컬럼을 선택하고 numpy 배열로 변환
X = fish.loc[fish.Species.isin(['Bream', 'Smelt']), ['Weight', 'Length']].values
X[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [14]:
X.shape  #> (49, 2) shape의 2차원 배열

(49, 2)

# 데이터와 타겟을 훈련 셋/테스트 셋으로 분리

`X`: 특성 행렬(feature matrix), `y`: 타겟 벡터(target vector)

## sequential sampling

In [16]:
# 훈련 셋과 테스트 셋을 7:3 비율로 나누기 위해서
num_trains = 35  # 훈련 셋 개수

In [17]:
X_train = X[:num_trains]  # 훈련 셋
X_test = X[num_trains:]   # 테스트 셋
y_train = y[:num_trains]  # 훈련 레이블
y_test = y[num_trains:]   # 테스트 레이블

In [18]:
X_train.shape, X_test.shape

((35, 2), (14, 2))

In [19]:
y_train.shape, y_test.shape

((35,), (14,))

데이터가 무작위로 섞여있지 않고 어종에 대해서 정렬된 상태였기 때문에, 훈련 셋/레이블에는 도미(Bream)만 선택되고, 테스트 셋/레이블에는 빙어(Smelt)만 선택됨.
---> **샘플링 편향(sampling bias)**

In [20]:
np.unique(y_train, return_counts=True)

(array(['Bream'], dtype=object), array([35]))

In [21]:
np.unique(y_test, return_counts=True)

(array(['Smelt'], dtype=object), array([14]))

# KNN 모델 훈련, 평가

In [22]:
knn_clf = KNeighborsClassifier()  # KNN 모델 생성

In [23]:
knn_clf.fit(X=X_train, y=y_train)  # 모델 훈련, fitting

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [24]:
# 훈련 셋 예측 결과
train_pred = knn_clf.predict(X=X_train)
train_pred

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream'],
      dtype=object)

In [25]:
train_acc = np.mean(y_train == train_pred)  # 훈련 셋에서의 정확도
train_acc

1.0

In [26]:
# 테스트 셋 예측 결과
test_pred = knn_clf.predict(X=X_test)
test_pred

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream'],
      dtype=object)

In [29]:
np.mean(y_test == test_pred)  # 테스트 셋에서의 정확도

0.0

`sklearn.model_selection_train_test_split` 함수를 사용한 훈련/테스트 셋 sequential sampling

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  # 특성 행렬, 타겟 벡터
                                                    train_size=35,  # 훈련 셋 원소 개수
                                                    shuffle=False)  # 데이터를 랜덤하게 섞을 지 말 지 결정

In [31]:
X_train.shape, X_test.shape

((35, 2), (14, 2))

In [32]:
y_train.shape, y_test.shape

((35,), (14,))

In [33]:
y_train

array(['Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream',
       'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream', 'Bream'],
      dtype=object)

In [34]:
y_test

array(['Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt',
       'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt', 'Smelt'],
      dtype=object)